In [1]:
from pathlib import Path
import numpy as np
from funcionesTP import *


In [2]:


def cargarDataset(carpeta: Path):
    pathCats = carpeta.joinpath('cats/efficientnet_b3_embeddings.npy')
    pathDogs = carpeta.joinpath('dogs/efficientnet_b3_embeddings.npy')

    embeddingsCats = np.load(pathCats)
    embeddingsDogs = np.load(pathDogs)

    embeddings = np.concatenate((embeddingsCats, embeddingsDogs), axis=1)
    _, m = embeddings.shape

    Y = np.zeros((2,m))
    for i in range(0, int(m/2)):
        Y[0,i] = 1

    for i in range(int(m/2), m):
        Y[1,i] = 1

    return embeddings, Y

def cargarDatasetCompleto():
    Xt, Yt = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/train'))
    Xv, Yv = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))

    return Xt, Yt, Xv, Yv



In [ ]:
def test_Cholesky():
    Xt, Yt, Xv, Yv = cargarDatasetCompleto()

    print('dataset cargado!')
    
    W = pinvEcuacionesNormales(Xt, Yt)
    return W

W = test_Cholesky()





In [ ]:
np.save('W_cholesky.npy', W)

In [4]:
def test_QRHH():
    Xt, Yt, Xv, Yv = cargarDatasetCompleto()

    print('dataset cargado!')
    Q, R = np.linalg.qr(traspuesta(Xt))
    print("calculo qr")
    W = qrFCN(Q, R, Yt)

    return W

W = test_QRHH()

dataset cargado!
calculo qr


100%|██████████| 2000/2000 [29:56<00:00,  1.11it/s]


In [6]:
np.save('W_qr.npy', W)